# WaveGlow Model inferencing

by Hyungon Ryu | Sr. Solution Architect


I'll demonstrate real-time inferencing WaveGlow

## allocate GPU

check GPU is available in your notebook


 in **[EDIT]** Menu> Select **[Notebook Settings]** > check ** [ GPU ]**

In [73]:
!nvidia-smi

Thu Nov  8 12:31:05 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Devops

for WaveGlow, we need **pytorch 0.4.0** with CUDA 9.0
**(TODO : I'll update pytorch 0.4.1+ in CUDA 10)**

in below script, you will update cuda 9.0

In [4]:
%%time
!rm -rf cuda-repo*
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!apt-get install dirmngr
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install  -y --no-install-recommends  \
 cuda-core-9-0 \
 cuda-cublas-9-0 cuda-cublas-dev-9-0 cuda-cudart-9-0 cuda-cudart-dev-9-0 \
 cuda-cufft-9-0 cuda-cufft-dev-9-0 cuda-curand-9-0 cuda-curand-dev-9-0 \
 cuda-cusolver-9-0 cuda-cusolver-dev-9-0 cuda-cusparse-9-0 \
 cuda-cusparse-dev-9-0 \
 cuda-libraries-9-0 cuda-libraries-dev-9-0 \
 cuda-misc-headers-9-0 cuda-npp-9-0 cuda-npp-dev-9-0 \
 cuda-nvgraph-9-0 cuda-nvgraph-dev-9-0 cuda-nvml-dev-9-0 cuda-nvrtc-9-0 \
 cuda-nvrtc-dev-9-0
#!rm -rf cuda-repo*
#!rm -rf wget-log*

--2018-11-08 10:37:07--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.189.146
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.189.146|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?2A6zw8fVfPOrL6eMiAgkjqv9CZHvNMMDLEpJeyeB_c9BaZMOyKQBnBFuxHhEFBPidQ8n1XXRv5C8u4RxOUY4K3QKMccY90M5ykb8jFhjXSsxSgsXeEONOc8SpKQ9GRMwLyNr-nC2Qao5D__tPC3SyeYeJgh1WD6L8pl8r1PWGH1T-4XjCCs-Du06PkxJGYfLVzJ_2VHnyzQ4EEnJPHNv [following]
--2018-11-08 10:37:08--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?2A6zw8fVfPOrL6eMiAgkjqv9CZHvNMMDLEpJeyeB_c9BaZMOyKQBnBFuxHhEFBPidQ8n1XXRv5C8u4RxOUY4K3QKMccY90M5ykb8jFhjXSsxSg

### install  pytorch 0.4.0

for **Tacotron2** and **WaveGlow** dependency, we need **librosa and other libraries 

In [5]:
!pip install -q torch==0.4.0

tcmalloc: large alloc 1073750016 bytes == 0x5bb3c000 @  0x7f80388442a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
!pip install -q  matplotlib==2.1.0 

In [8]:
!pip install numpy inflect librosa scipy tensorboardX Unidecode pillow

  Using cached https://files.pythonhosted.org/packages/6e/1b/6b9b48323b714b5f66dbea2bd5d4166c4f99d908bc31d5307d14083aa9a2/inflect-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/b4/5b411f19de48f8fc1a0ff615555aa9124952e4156e94d4803377e50cfa4c/librosa-0.6.2.tar.gz
    100% |████████████████████████████████| 71kB 2.5MB/s 
    100% |████████████████████████████████| 235kB 7.8MB/s 
    100% |████████████████████████████████| 327kB 11.1MB/s 
    100% |████████████████████████████████| 3.2MB 9.8MB/s 
    100% |████████████████████████████████| 16.1MB 2.5MB/s 
  Running setup.py bdist_wheel for librosa ... - \ done
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
  Running setup.py bdist_wheel for audioread ... - done
  Stored in directory: /root/.cache/pip/wheels/53/02/90/7b5c4081b7470c550ab605f600bad237dde12a6b8999b11f50
  Running setup.py bdist_wheel for resampy ... - \ done
  Stored in 

## clone WaveGlow Model
I'm using my repository. I strongly recommend to use NVIDIA's official git repository 


In [9]:
#!git clone https://github.com/NVIDIA/waveglow.git
!git clone https://github.com/yhgon/waveglow.git

Cloning into 'waveglow'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [12]:
!ls -alh waveglow

total 464K
drwxr-xr-x 4 root root 4.0K Nov  8 10:52 .
drwxr-xr-x 1 root root 4.0K Nov  8 10:52 ..
-rwxr-xr-x 1 root root  881 Nov  8 10:52 config.json
-rw-r--r-- 1 root root 7.3K Nov  8 10:52 distributed.py
drwxr-xr-x 8 root root 4.0K Nov  8 10:52 .git
-rw-r--r-- 1 root root   84 Nov  8 10:52 .gitmodules
-rw-r--r-- 1 root root 5.9K Nov  8 10:52 glow_old.py
-rw-r--r-- 1 root root  13K Nov  8 10:52 glow.py
-rw-r--r-- 1 root root 3.5K Nov  8 10:52 inference.py
-rw-r--r-- 1 root root 5.7K Nov  8 10:52 mel2samp.py
-rwxr-xr-x 1 root root 2.7K Nov  8 10:52 README.md
drwxr-xr-x 2 root root 4.0K Nov  8 10:52 tacotron2
-rw-r--r-- 1 root root 7.3K Nov  8 10:52 train.py
-rwxr-xr-x 1 root root 382K Nov  8 10:52 waveglow_logo.png


## install tacotron2

to match audio file preprocessing, we need to  use tacotron2.
you could update tacotron2 repository in submodule

In [14]:
%%bash
pwd
cd waveglow
git submodule init
git submodule update
ls -lah
ls -alh tacotron2

/content
Submodule path 'tacotron2': checked out 'fc0cf6a89a47166350b65daa1beaa06979e4cddf'
total 464K
drwxr-xr-x 4 root root 4.0K Nov  8 10:52 .
drwxr-xr-x 1 root root 4.0K Nov  8 10:52 ..
-rwxr-xr-x 1 root root  881 Nov  8 10:52 config.json
-rw-r--r-- 1 root root 7.3K Nov  8 10:52 distributed.py
drwxr-xr-x 9 root root 4.0K Nov  8 10:55 .git
-rw-r--r-- 1 root root   84 Nov  8 10:52 .gitmodules
-rw-r--r-- 1 root root 5.9K Nov  8 10:52 glow_old.py
-rw-r--r-- 1 root root  13K Nov  8 10:52 glow.py
-rw-r--r-- 1 root root 3.5K Nov  8 10:52 inference.py
-rw-r--r-- 1 root root 5.7K Nov  8 10:52 mel2samp.py
-rwxr-xr-x 1 root root 2.7K Nov  8 10:52 README.md
drwxr-xr-x 4 root root 4.0K Nov  8 10:55 tacotron2
-rw-r--r-- 1 root root 7.3K Nov  8 10:52 train.py
-rwxr-xr-x 1 root root 382K Nov  8 10:52 waveglow_logo.png
total 1.2M
drwxr-xr-x 4 root root 4.0K Nov  8 10:55 .
drwxr-xr-x 4 root root 4.0K Nov  8 10:52 ..
-rw-r--r-- 1 root root 2.6K Nov  8 10:55 audio_processing.py
-rw-r--r-- 1 root root 

Submodule 'tacotron2' (http://github.com/NVIDIA/tacotron2) registered for path 'tacotron2'
Cloning into '/content/waveglow/tacotron2'...


In [15]:
!pwd

/content


## Download Weight 

there are two options.  I strongly recommend to copy to google drive to reuse. 

 - colab drive
 
  - step1. copy to google drive
  - step2. mount Google drive
  - step3. copy to local disk in COLAB

- direct download
 
 -step1. download file to COLAB disk
 

 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

I copy the weight files and mel files in COLAB_data folder.
change right folder 

In [21]:
%%time
!ls -alh   "drive/My Drive/COLAB_data"

total 2.0G
dr-x------ 2 root root 4.0K Jul 28  2016 CelebA
dr-x------ 2 root root 4.0K Nov 11  2017 celeba-hq-deltas
-r-------- 2 root root 1.5M Nov  8 00:41 mel_spectrograms.zip
drwx------ 2 root root 4.0K Oct  4 10:39 segan
-r-------- 2 root root 2.0G Nov  8 03:51 waveglow_old.pt
CPU times: user 41.6 ms, sys: 11.2 ms, total: 52.8 ms
Wall time: 4.29 s


It takes 32 sec to copy 2GB file from google drive to COLAB  

In [22]:
%%time
!cp -rf "drive/My Drive/COLAB_data/waveglow_old.pt" .

CPU times: user 167 ms, sys: 21.4 ms, total: 188 ms
Wall time: 32.2 s


In [23]:
%%time
!cp -rf "drive/My Drive/COLAB_data/mel_spectrograms.zip" .

CPU times: user 32.7 ms, sys: 12.9 ms, total: 45.5 ms
Wall time: 3.66 s


In [24]:
!pwd
!ls -alh

/content
total 3.2G
drwxr-xr-x 1 root root 4.0K Nov  8 11:11  .
drwxr-xr-x 1 root root 4.0K Nov  8 10:33  ..
drwxr-xr-x 4 root root 4.0K Nov  6 18:04  .config
-rw-r--r-- 1 root root 1.2G Sep 23  2017  cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
drwx------ 3 root root 4.0K Nov  8 11:05  drive
-r-------- 1 root root 1.5M Nov  8 11:11  mel_spectrograms.zip
drwxr-xr-x 2 root root 4.0K Nov  6 18:16  sample_data
-rw-r--r-- 1 root root 133K Nov  8 10:58 'view?usp=sharing'
drwxr-xr-x 4 root root 4.0K Nov  8 10:52  waveglow
-r-------- 1 root root 2.0G Nov  8 11:09  waveglow_old.pt


##  Mel Spectrum
(TODO ) add live Mel Spectrum Generation from text with Tacotron2

### decompress Mel files

In [32]:

!unzip mel_spectrograms.zip

Archive:  mel_spectrograms.zip
  inflating: mel_spectrograms/LJ001-0153.wav.pt  
  inflating: mel_spectrograms/LJ001-0096.wav.pt  
  inflating: mel_spectrograms/LJ001-0094.wav.pt  
  inflating: mel_spectrograms/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/mel_spectrograms/
  inflating: __MACOSX/mel_spectrograms/._.DS_Store  
  inflating: mel_spectrograms/LJ001-0079.wav.pt  
  inflating: mel_spectrograms/LJ001-0051.wav.pt  
  inflating: mel_spectrograms/LJ001-0063.wav.pt  
  inflating: mel_spectrograms/LJ001-0173.wav.pt  
  inflating: mel_spectrograms/LJ001-0102.wav.pt  
  inflating: mel_spectrograms/LJ001-0015.wav.pt  
  inflating: mel_spectrograms/LJ001-0072.wav.pt  


I found file name have some issue because original file is compressed in MACOS.
so eliminate the folder  and decompress agin excluding  MACOS related files  

https://github.com/NVIDIA/waveglow/issues/3

In [71]:
!pwd
!mv mel_spectrograms aaa
!ls 
!rm -rf aaa
!unzip mel_spectrograms.zip -d /content

/content
 aaa						      mel_spectrograms.zip
 a.pt						      sample_data
 cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb  'view?usp=sharing'
 drive						      waveglow
 LJ001-0051.wav.pt				      waveglow_old.pt
 __MACOSX
Archive:  mel_spectrograms.zip
   creating: /content/mel_spectrograms/
  inflating: /content/mel_spectrograms/LJ001-0153.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0096.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0094.wav.pt  
  inflating: /content/mel_spectrograms/.DS_Store  
replace /content/__MACOSX/mel_spectrograms/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: /content/mel_spectrograms/LJ001-0079.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0051.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0063.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0173.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0102.wav.pt  
  inflating: /content/mel_spectrograms/LJ001-0015.wav.pt  
  inflating: /content/m

if you use your own repository and modified code,  update  

In [0]:
%%bash
cd waveglow
git pull

# Inferencing

it takes few seconds  to load weight files(2GB) and synthesize voice.

In [72]:
%%bash
pwd
cd waveglow

python inference.py  -f <(ls /content/mel_spectrograms/*.pt)  -w /content/waveglow_old.pt -o . -s 0.6


/content
./LJ001-0015.wav_synthesis.wav
./LJ001-0051.wav_synthesis.wav
./LJ001-0063.wav_synthesis.wav
./LJ001-0072.wav_synthesis.wav
./LJ001-0079.wav_synthesis.wav
./LJ001-0094.wav_synthesis.wav
./LJ001-0096.wav_synthesis.wav
./LJ001-0102.wav_synthesis.wav
./LJ001-0153.wav_synthesis.wav
./LJ001-0173.wav_synthesis.wav


## listen voice 
You could  compare original LJSpeech dataset

In [77]:
import librosa.display
import IPython
from IPython.display import Audio
IPython.display.display(Audio('/content/waveglow/LJ001-0015.wav_synthesis.wav', rate=22050))
IPython.display.display(Audio('/content/waveglow/LJ001-0051.wav_synthesis.wav', rate=22050))

## 

### (TODO) real-time inferencing with Tacotron Weight + WaveGlow model.
 - Step1 DevOps
 
 - Step2. clone Model and download weight files
 
 - Step3 load Model param
 
  -Tacotron2 weight
  - WaveGlow weight

- Step4. input text 

- Step5. Generate Mel 
- Step6. Generate Wav
- Step7. display wavefile
